In [1]:
#importing the dependencies 
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import sys

Using TensorFlow backend.


In [31]:
# load ascii text and covert to lowercase
filename = "wonderland.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

In [32]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [33]:
# here we can see all the unique charecters in the book
print(chars)

['\n', ' ', '!', '(', ')', '*', ',', '-', '.', '0', '3', ':', ';', '?', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '‘', '’', '“', '”']


In [34]:
#here we can see all the unique charecter are mapped to a unique number
print(char_to_int)

{' ': 1, '?': 13, '-': 7, '[': 14, 'n': 30, '!': 2, '.': 8, 'b': 18, '_': 16, 'i': 25, 'h': 24, '’': 44, ']': 15, 'v': 38, 's': 35, '\n': 0, 'p': 32, 'e': 21, 'm': 29, 'l': 28, 'c': 19, 'x': 40, 'd': 20, 'u': 37, 'f': 22, '*': 5, 'a': 17, 'y': 41, '‘': 43, 't': 36, '0': 9, '(': 3, '“': 45, 'o': 31, ';': 12, '3': 10, 'r': 34, 'j': 26, '”': 46, 'k': 27, 'w': 39, ',': 6, 'z': 42, ')': 4, 'g': 23, 'q': 33, ':': 11}


In [35]:
n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)

Total Characters:  144515
Total Vocab:  47


In [36]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    #print('seq in : ', seq_in)
    seq_out = raw_text[i + seq_length]
    #print('seq out : ', seq_out)
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)

Total Patterns:  144415


In [37]:
print(dataX)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [38]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [32]:

# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])

In [33]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 256)               264192    
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 47)                12079     
Total params: 276,271
Trainable params: 276,271
Non-trainable params: 0
_________________________________________________________________


In [34]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [35]:
# finally training the model
model.fit(X, y, epochs=20, batch_size=2000, callbacks=callbacks_list)

Epoch 1/20
144415/144415 [==============================] - 1075s 7ms/step - loss: 3.1562 - acc: 0.1636
Epoch 2/20
144415/144415 [==============================] - 1058s 7ms/step - loss: 3.0570 - acc: 0.1730
Epoch 3/20
144415/144415 [==============================] - 1056s 7ms/step - loss: 3.0498 - acc: 0.1734
Epoch 4/20
144415/144415 [==============================] - 1059s 7ms/step - loss: 3.0419 - acc: 0.1734
Epoch 5/20
144415/144415 [==============================] - 1055s 7ms/step - loss: 3.0190 - acc: 0.1731
Epoch 6/20
144415/144415 [==============================] - 1055s 7ms/step - loss: 2.9521 - acc: 0.1860
Epoch 7/20
144415/144415 [==============================] - 1056s 7ms/step - loss: 2.9077 - acc: 0.2017
Epoch 8/20
144415/144415 [==============================] - 1108s 8ms/step - loss: 2.8733 - acc: 0.2151
Epoch 9/20
144415/144415 [==============================] - 1085s 8ms/step - loss: 2.8447 - acc: 0.2234
Epoch 10/20
144415/144415 [==============================] - 106

In [ ]:
# training the model for more 40 epochs 
model.fit(X, y, epochs=40, batch_size=2500, callbacks=callbacks_list)

Epoch 1/40
 25000/144415 [====>.........................] - ETA: 14:58 - loss: 2.6162 - acc: 0.2676

In [36]:
model.save('model.h5')


In [28]:
from keras.models import load_model
model = load_model('saved weights small network/weights-improvement-20-2.6334.hdf5')

In [22]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [39]:
#generating the text 


# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
#print(dataX)
start = 0
pattern = dataX[start]
print ("Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

# generate characters
for i in range(1000):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print ("\nDone.")

Seed:
" *** start of this project gutenberg ebook alice’s adventures in wonderland ***

alice’s adventures i "
o the  a  a  ao the  an the  an the  a  a  a  a  a  a  a  a                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

here we can see that the model performs very badly. an educated guess for this is that the model is too simple and is also trained for smaller epochs 

In [15]:
# training larger model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])

In [16]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 100, 256)          264192    
_________________________________________________________________
dropout_3 (Dropout)          (None, 100, 256)          0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 47)                12079     
Total params: 801,583
Trainable params: 801,583
Non-trainable params: 0
_________________________________________________________________


In [17]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(X, y, epochs=10, batch_size=1300, callbacks=callbacks_list)

Epoch 1/10
144415/144415 [==============================] - 3209s 22ms/step - loss: 3.1091 - acc: 0.1666
Epoch 2/10
144415/144415 [==============================] - 3174s 22ms/step - loss: 3.0332 - acc: 0.1735
Epoch 3/10
144415/144415 [==============================] - 3175s 22ms/step - loss: 2.9269 - acc: 0.1947
Epoch 4/10
144415/144415 [==============================] - 3177s 22ms/step - loss: 2.8118 - acc: 0.2299
Epoch 5/10
144415/144415 [==============================] - 3176s 22ms/step - loss: 2.7228 - acc: 0.2488
Epoch 6/10
144415/144415 [==============================] - 3174s 22ms/step - loss: 2.6504 - acc: 0.2617
Epoch 7/10
144415/144415 [==============================] - 3173s 22ms/step - loss: 2.5766 - acc: 0.2765
Epoch 8/10
144415/144415 [==============================] - 3214s 22ms/step - loss: 2.5078 - acc: 0.2922
Epoch 9/10
144415/144415 [==============================] - 3140s 22ms/step - loss: 2.4480 - acc: 0.3052
Epoch 10/10
144415/144415 [============================

In [18]:
#training the model for another round of 10 epochs 
model.fit(X, y, epochs=10, batch_size=1300, callbacks=callbacks_list)

Epoch 1/10
144415/144415 [==============================] - 3275s 23ms/step - loss: 2.3322 - acc: 0.3312
Epoch 2/10
144415/144415 [==============================] - 3333s 23ms/step - loss: 2.2832 - acc: 0.3450
Epoch 3/10
144415/144415 [==============================] - 3205s 22ms/step - loss: 2.2351 - acc: 0.3579
Epoch 4/10
144415/144415 [==============================] - 3310s 23ms/step - loss: 2.1945 - acc: 0.3696
Epoch 5/10
144415/144415 [==============================] - 3150s 22ms/step - loss: 2.1582 - acc: 0.3801
Epoch 6/10
144415/144415 [==============================] - 3139s 22ms/step - loss: 2.1219 - acc: 0.3895
Epoch 7/10
144415/144415 [==============================] - 3180s 22ms/step - loss: 2.0907 - acc: 0.3963
Epoch 8/10
144415/144415 [==============================] - 3342s 23ms/step - loss: 2.0630 - acc: 0.4049
Epoch 9/10
144415/144415 [==============================] - 3251s 23ms/step - loss: 2.0331 - acc: 0.4122
Epoch 10/10
144415/144415 [============================

In [24]:
#generating the text
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print ("\nDone.")

Seed:
" do in an offended tone, ‘was, that
the best thing to get us dry would be a caucus-race.’

‘what is a "
 larter ’ said the ming. ‘i dan to the thing the was to the thing the was to the thing the was a little said the was oot a little sabbit with the was oot a little sabbit with the was oot a little sabbit with the was oot a little sabbit with the was oot a little sabbit with the was oot a little sabbit with the was oot a little sabbit with the was oot a little sabbit with the was oot a little sabbit with the was oot a little sabbit with the was oot a little sabbit with the was oot a little sabbit with the was oot a little sabbit with the was oot a little sabbit with the was oot a little sabbit with the was oot a little sabbit with the was oot a little sabbit with the was oot a little sabbit with the was oot a little sabbit with the was oot a little sabbit with the was oot a little sabbit with the was oot a little sabbit with the was oot a little sabbit with the was oot a little 

We can see that generally there are fewer spelling mistakes and the text looks more realistic, but is still quite nonsensical.
try fitting a more complex network or try training this network for longer epochs in order to see considerable differences